In [1]:
!pip install rapidfuzz easyocr pandas requests pillow matplotlib numpy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 15.0 MB/s eta 0:00:0000:0100:01


In [2]:
!sudo apt-get install build-essential cmake git pkg-config libgtk-3-dev \
    libavcodec-dev libavformat-dev libswscale-dev libv4l-dev libxvidcore-dev \
    libx264-dev libjpeg-dev libpng-dev libtiff-dev gfortran openexr \
    libatlas-base-dev python3-dev python3-numpy libtbb2 libtbb-dev \
    libdc1394-22-dev libopenexr-dev libgstreamer-plugins-base1.0-dev \
    libgstreamer1.0-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package libdc1394-22-dev


In [3]:
!git clone https://github.com/opencv/opencv.git
!git clone https://github.com/opencv/opencv_contrib.git

Cloning into 'opencv'...
remote: Enumerating objects: 336025, done.
remote: Counting objects: 100% (798/798), done.
remote: Compressing objects: 100% (655/655), done.
remote: Total 336025 (delta 293), reused 514 (delta 120), pack-reused 335227 (from 1)
Receiving objects: 100% (336025/336025), 526.16 MiB | 34.91 MiB/s, done.
Resolving deltas: 100% (234282/234282), done.
Updating files: 100% (7566/7566), done.
Cloning into 'opencv_contrib'...
remote: Enumerating objects: 41542, done.
remote: Counting objects: 100% (1301/1301), done.
remote: Compressing objects: 100% (938/938), done.
remote: Total 41542 (delta 486), reused 911 (delta 288), pack-reused 40241 (from 1)
Receiving objects: 100% (41542/41542), 149.98 MiB | 33.67 MiB/s, done.
Resolving deltas: 100% (25618/25618), done.


In [11]:

%cd /kaggle/working/build

!cmake -DOPENCV_EXTRA_MODULES_PATH=/kaggle/working/opencv_contrib/modules  \
       -DBUILD_SHARED_LIBS=OFF \
       -DBUILD_TESTS=OFF \
       -DBUILD_PERF_TESTS=OFF \
       -DBUILD_EXAMPLES=OFF \
       -DWITH_OPENEXR=OFF \
       -DWITH_CUDA=ON \
       -DWITH_CUBLAS=ON \
       -DWITH_CUDNN=ON \
       -DOPENCV_DNN_CUDA=ON \
       /kaggle/working/opencv

!make -j8 install


/kaggle/working/build
cmake: /opt/conda/lib/libcurl.so.4: no version information available (required by cmake)
-- 'Release' build type is used by default. Use CMAKE_BUILD_TYPE to specify build type (Release or Debug)
-- The CXX compiler identification is GNU 11.4.0
-- The C compiler identification is GNU 11.4.0
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detected processor: x86_64
-- Found PythonInterp: /opt/conda/bin/python3 (found suitable version "3.10.14", minimum required is "3.2") 
-- Could NOT find PythonLibs (missing: PYTHON_LIBRARIES PYTHON_INCLUDE_DIRS) (Required is exact version "3.10.14")
-- Looking for ccache - 

In [6]:
!pip install torch --index-url https://download.pytorch.org/whl/cu118


Looking in indexes: https://download.pytorch.org/whl/cu118


In [15]:
import torch
print(torch.cuda.is_available())
import easyocr

reader = easyocr.Reader(['en'], gpu=True)
print("Reader using GPU:", reader.device)


True


/opt/conda/lib/python3.10/site-packages/easyocr/detection.py:85: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(copyStateDict(torch.load(trained_model, ma

Reader using GPU: cuda


/opt/conda/lib/python3.10/site-packages/easyocr/recognition.py:182: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=d

In [14]:
import cv2
print("CUDA Enabled Device Count:", cv2.cuda.getCudaEnabledDeviceCount())

CUDA Enabled Device Count: 0


In [ ]:
import re
from rapidfuzz import process
import easyocr
import pandas as pd
import requests
from PIL import Image
from io import BytesIO
import cv2
from matplotlib import pyplot as plt
import numpy as np
from utils import download_image
import csv

In [17]:
# Define the dictionary with full forms and abbreviations
unit_fullform_to_abbreviation = {
    'centimetre': ['cm', 'centimeter', 'centimetres', 'centimeters'],
    'millimetre': ['mm', 'millimeter', 'millimetres', 'millimeters'],
    'metre': ['m', 'meter', 'meters', 'metres'],
    'inch': ['in', 'inches', '"'],
    'foot': ['ft', 'feet', "'"],
    'yard': ['yd', 'yards'],
    'gram': ['g', 'gm', 'gms', 'grams'],
    'kilogram': ['kg', 'kgs', 'kilograms'],
    'milligram': ['mg', 'mgs', 'milligrams'],
    'microgram': ['mcg', 'μg', 'micrograms'],
    'ounce': ['oz', 'ounces'],
    'pound': ['lb', 'lbs', 'pounds'],
    'ton': ['t', 'tonne', 'tonnes', 'tons'],
    'volt': ['v', 'volts'],
    'kilovolt': ['kv', 'kilovolts'],
    'millivolt': ['mv', 'millivolts'],
    'watt': ['w', 'watts'],
    'kilowatt': ['kw', 'kilowatts'],
    'litre': ['l', 'liters', 'litre', 'litres'],
    'millilitre': ['ml', 'milliliters', 'millilitres', 'mls'],
    'centilitre': ['cl', 'centiliters', 'centilitres'],
    'decilitre': ['dl', 'deciliters', 'decilitres'],
    'cubic foot': ['cu ft', 'ft³'],
    'cubic inch': ['cu in', 'in³'],
    'fluid ounce': ['fl oz', 'fluid ounces'],
    'gallon': ['gal', 'gallons'],
    'imperial gallon': ['imp gal', 'imperial gallons'],
    'pint': ['pt', 'pints'],
    'quart': ['qt', 'quarts'],
    'microlitre': ['ul', 'μl', 'microlitres', 'microliters'],
    'cup': ['cup', 'cups']
}

# Unit conversion factors to smallest units (e.g., mm for length, mg for weight, mv for voltage)
conversion_factors_to_smallest = {
    'millimetre': 1, 'centimetre': 10, 'metre': 1000, 'inch': 25.4, 'foot': 304.8, 'yard': 914.4,
    'milligram': 1, 'gram': 1000, 'kilogram': 1000000, 'microgram': 0.001, 'ounce': 28349.5, 'pound': 453592, 'ton': 1000000000,
    'millivolt': 1, 'volt': 1000, 'kilovolt': 1000000,
    'millilitre': 1, 'litre': 1000, 'centilitre': 10, 'decilitre': 100,
    'fluid ounce': 29.5735, 'gallon': 3785.41, 'pint': 473.176, 'quart': 946.353,
    'microlitre': 0.001, 'cubic inch': 16.3871, 'cubic foot': 28316.8,
    'watt': 1, 'kilowatt': 1000
}

# Classify units into categories
unit_categories = {
    'length': {'millimetre', 'centimetre', 'metre', 'inch', 'foot', 'yard'},
    'weight': {'milligram', 'gram', 'kilogram', 'microgram', 'ounce', 'pound', 'ton'},
    'voltage': {'millivolt', 'volt', 'kilovolt'},
    'volume': {'millilitre', 'litre', 'centilitre', 'decilitre', 'fluid ounce', 'gallon', 'pint', 'quart', 'microlitre', 'cubic inch', 'cubic foot'},
    'wattage': {'watt', 'kilowatt'}
}

# Flatten the unit list for fuzzy matching
all_units = [abbrev for unit_list in unit_fullform_to_abbreviation.values()
             for abbrev in unit_list]


In [32]:
# Function to extract numbers and their (possibly misspelled) units
def extract_numbers_with_units(text):
    # Regex to match numbers followed by potential unit strings
    pattern = r'(\d+(\.\d+)?)(\s*)([a-zA-Z]+)'
    # pattern = r'(\d+((\.)+\d+)?)(\s*)([a-zA-Z]+)'
    matches = re.findall(pattern, text)

    results = []
    # Iterate over all matches
    for match in matches:
        number = match[0]  # The number (either integer or decimal)
        # The potential unit (word following the number)
        unit_candidate = match[3]
        # Fuzzy match the unit candidate to the closest valid unit
        results_fuzzy = process.extractOne(
            unit_candidate, all_units, score_cutoff=95)
        if results_fuzzy:
            matched_unit = results_fuzzy
            # If a match is found with a reasonable score, append the result
            results.append(f"{number} {matched_unit[0]}")

    return results


In [19]:
'''PART 2'''
# Separating the numbers and units
def separate_numbers_and_units(output):
    numbers_list = []
    units_list = []
    for item in output:
        match = re.match(r'([0-9.]+)\s*(\w+)', item)
        if match:
            number, unit = match.groups()
            numbers_list.append(number)
            units_list.append(unit)

    return numbers_list, units_list

In [20]:
'''PART 3'''

# Create a set of all allowed full forms
allowed_units = set(unit_fullform_to_abbreviation.keys())


# Function to get full unit or blank space
def get_full_unit(unit_input):
    # Use regex to convert the input to lowercase and strip any extra spaces
    unit_input = re.sub(r'\s+', ' ', unit_input.lower().strip())

    # Check if the input matches any abbreviation or full form
    for full_unit, abbreviations in unit_fullform_to_abbreviation.items():
        if unit_input == full_unit or unit_input in abbreviations:
            return full_unit

    # If no match, return blank
    return ""


def convert_to_full_unit(input_text):
    final_result = []
    result = []
    numbers = []
    units = []
    # Getting the initial output
    output = extract_numbers_with_units(input_text)
    # print("Extracted values:", output)

    numbers, units = separate_numbers_and_units(output)

    # print("Numbers:", numbers)
    # print("Units:", units)

    # Getting full forms of units
    for unit in units:
        unit_fullform = get_full_unit(unit)
        result.append(unit_fullform)

    # Combine numbers and full units in the desired format
    final_result = [f"{num} {unit}" for num, unit in zip(numbers, result)]
    return final_result, numbers, result

In [30]:
def categorize_values(numbers, result):
    categorized_values = {
        'length': [],
        'weight': [],
        'voltage': [],
        'volume': [],
        'wattage': []
    }

    # full_units, _ = convert_to_full_unit(', '.join(input_list))

    # numbers, _ = separate_numbers_and_units(input_list)

    for num, unit in zip(numbers, result):
        for category, unit_set in unit_categories.items():
            if unit in unit_set:
                converted_value = float(
                    num) * conversion_factors_to_smallest[unit]
                categorized_values[category].append(
                    (converted_value, f"{float(num)} {unit}"))

    return categorized_values

def get_entity_value(entity_value, categorized_values):
    # Return based on entity type
    if entity_value == 'item_weight' or entity_value == 'maximum_weight_recommendation':
        if categorized_values['weight']:
            # Largest weight
            return max(categorized_values['weight'], key=lambda x: x[0])[1]
    elif entity_value == 'voltage':
        if categorized_values['voltage']:
            # Largest voltage
            return max(categorized_values['voltage'], key=lambda x: x[0])[1]
    elif entity_value == 'item_volume':
        if categorized_values['volume']:
            # Largest volume
            return max(categorized_values['volume'], key=lambda x: x[0])[1]
    elif entity_value == 'wattage':
        if categorized_values['wattage']:
            # Largest wattage
            return max(categorized_values['wattage'], key=lambda x: x[0])[1]
    elif entity_value in ['width', 'depth', 'height']:
        if categorized_values['length']:
            if entity_value == 'width':
                # Smallest length for width
                return min(categorized_values['length'], key=lambda x: x[0])[1]
            else:
                # Largest length for depth/height
                return max(categorized_values['length'], key=lambda x: x[0])[1]

    return None  # If no valid match is found

In [31]:
def easyocr_model(image):
    reader = easyocr.Reader(['en'], gpu = True)
    result = reader.readtext(image)
    text = ' '.join([item[1] for item in result])
    return text


# def grayscale(image):
#     return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

def grayscale(image):
    # Use OpenCV CUDA for faster grayscale conversion if available
    if cv2.cuda.getCudaEnabledDeviceCount() > 0:
        image_gpu = cv2.cuda_GpuMat()
        image_gpu.upload(image)
        gray_gpu = cv2.cuda.cvtColor(image_gpu, cv2.COLOR_BGR2GRAY)
        gray_image = gray_gpu.download()
    else:
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return gray_image


# Function to download image and load it in memory
def load_image_from_url(image_url):
    try:
        response = requests.get(image_url)
        if response.status_code == 200:
            image = Image.open(BytesIO(response.content))
            return image
        else:
            print(f"Failed to download image from {image_url}")
            return None
    except Exception as e:
        print(f"Error loading image: {e}")
        return None


def inversion_check(img):
    gray_image = grayscale(img)

    # Determine if background is light or dark by calculating the mean pixel intensity
    mean_intensity = np.mean(gray_image)
    print(mean_intensity)

    # Set a threshold to distinguish between dark and light backgrounds
    # If the mean intensity is lower than this value, we consider the background dark
    threshold_intensity = 127  # Midpoint in range [0, 255]

    if mean_intensity < threshold_intensity:
        # If background is dark (mean intensity is low), invert the image
        gray_image = cv2.bitwise_not(gray_image)
    return gray_image


# def sharpen_image(gray_image):
#     # Sharpening kernel
#     kernel = np.array([[0, -1, 0],
#                        [-1, 5, -1],
#                        [0, -1, 0]])

#     # Apply the sharpening kernel to the image
#     sharpened = cv2.filter2D(gray_image, -1, kernel)
#     return sharpened

def sharpen_image(gray_image):
    # Use OpenCV CUDA for sharpening if available
    if cv2.cuda.getCudaEnabledDeviceCount() > 0:
        gray_gpu = cv2.cuda_GpuMat()
        gray_gpu.upload(gray_image)

        # Sharpening kernel
        kernel = np.array([[0, -1, 0],
                           [-1, 5, -1],
                           [0, -1, 0]])

        kernel_gpu = cv2.cuda_GpuMat()
        kernel_gpu.upload(kernel)

        # Apply the sharpening kernel to the image on the GPU
        sharpened_gpu = cv2.cuda.filter2D(gray_gpu, -1, kernel)
        sharpened = sharpened_gpu.download()
    else:
        # Sharpen on the CPU if no CUDA support
        kernel = np.array([[0, -1, 0],
                           [-1, 5, -1],
                           [0, -1, 0]])
        sharpened = cv2.filter2D(gray_image, -1, kernel)
    return sharpened


In [ ]:
def process_csv(output_csv):
    if not os.path.isfile(output_csv):
            output_df.to_csv(output_csv, index=False, columns=['index_id', 'entity_name', 'extracted_text'])
        
    with open(output_csv, newline='') as infile, open(output_csv, mode='w', newline='') as outfile:
        reader = csv.DictReader(infile)
        writer = csv.writer(outfile)

        # Write the header for the output CSV
        writer.writerow(["index_id", "extracted_text"])

        index_id = 0
        for row in reader:
            image_url = row['image_link']
            image = download_image(image_url)
            if image is None:
                continue

            # Detect text
            detected_text = detect_text_east(image, east_model_path)

            # Write the result to the output CSV
            writer.writerow([index_id, detected_text])
            index_id += 1

In [33]:
# Load data from CSV file
# Update with the path to your input CSV
input_csv = '/kaggle/input/amazon-ml-2024/test.csv'
output_csv = '/kaggle/working/output.csv'  # Path to save the output CSV

data = pd.read_csv(input_csv)

# Create a list to store the results
results = []
batch_size = 5  # Set the interval to save results (e.g., every 5 images)

In [ ]:
import os
# Run OCR on each image link
for index, row in data.iterrows():
    image_link = row['image_link']
    entity_name = row['entity_name']

    print(f"Processing Image: {image_link}")

    # Load the image from the URL
    image = load_image_from_url(image_link)
    print(image)

    if image:
        try:
            # Convert PIL image to an OpenCV image (numpy array)
            image_np = np.array(image)

            # Pre-processing Images
            grayscale_image = inversion_check(image_np)
            sharpened_image = sharpen_image(grayscale_image)

            # Perform OCR
            extracted_text = easyocr_model(sharpened_image)
        except Exception as e:
            extracted_text = f"Error: {e}"
    else:
        extracted_text = "Image not available"

    extract, num, units = convert_to_full_unit(extracted_text)
    print(extract)
    categorized_values = categorize_values(num, units)
    final_val = get_entity_value(entity_name, categorized_values)

    # Append the result to the list
    results.append({
        'index_id': index,
        'entity_name': entity_name,
        'extracted_text': final_val
    })
    # Append results to the CSV every batch_size iterations
    if (index + 1) % batch_size == 0 or index == len(data) - 1:
        # Convert the current batch of results into a DataFrame
        output_df = pd.DataFrame(results)

        # Append to CSV if it exists, otherwise create it
        if not os.path.isfile(output_csv):
            output_df.to_csv(output_csv, index=False, columns=['index_id', 'prediction'])
        else:
            output_df.to_csv(output_csv, mode='a', header=False, index=False, columns=['index_id', 'extracted_text'])

        # Clear the results list after writing to CSV
        results = []

        print(f"Appended results to {output_csv}")

# Convert the results into a DataFrame
# output_df = pd.DataFrame(results)

# # Save the results to a new CSV file
# output_df.to_csv(output_csv, index=False, columns=[
#                  'index_id', 'entity_name', 'extracted_text'])

print(f"Results saved to {output_csv}")

Processing Image: https://m.media-amazon.com/images/I/110EibNyclL.jpg
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=500x500 at 0x7E54F00AE8F0>
253.842188
['5144 metre']
Processing Image: https://m.media-amazon.com/images/I/11TU2clswzL.jpg
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1000x65 at 0x7E53BC99EB60>
250.10558461538463
['42 centimetre', '200 centimetre']
Processing Image: https://m.media-amazon.com/images/I/11TU2clswzL.jpg
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1000x65 at 0x7E54F00AF0A0>
250.10558461538463
['42 centimetre', '200 centimetre']
Processing Image: https://m.media-amazon.com/images/I/11TU2clswzL.jpg
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1000x65 at 0x7E53BC1D6B60>
250.10558461538463
['42 centimetre', '200 centimetre']
Processing Image: https://m.media-amazon.com/images/I/11gHj8dhhrL.jpg
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1000x65 at 0x7E53C7CFBEE0>
250.10224615384615
['10.50 centimetre', '90 centi

/opt/conda/lib/python3.10/site-packages/easyocr/detection.py:85: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(copyStateDict(torch.load(trained_model, ma

['10.50 centimetre', '90 centimetre']
Processing Image: https://m.media-amazon.com/images/I/11gHj8dhhrL.jpg
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1000x65 at 0x7E53C7CF8370>
250.10224615384615
['10.50 centimetre', '90 centimetre']
Processing Image: https://m.media-amazon.com/images/I/11lshEUmCrL.jpg
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=500x500 at 0x7E53BC1D6A70>
250.4616
['7 pound', '4 metre']
Processing Image: https://m.media-amazon.com/images/I/21+i52HRW4L.jpg
<PIL.JpegImagePlugin.JpegImageFile image mode=L size=1024x1024 at 0x7E53C7CFABF0>
[]
Processing Image: https://m.media-amazon.com/images/I/21-LmSmehZL.jpg
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1024x1024 at 0x7E54F00AECE0>
251.31842041015625
['40 centimetre', '30 centimetre', '15 centimetre']
Appended results to /kaggle/working/output_easyocr.csv
Processing Image: https://m.media-amazon.com/images/I/213oP6n7jtL.jpg
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=200x200 a

/opt/conda/lib/python3.10/site-packages/easyocr/detection.py:85: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(copyStateDict(torch.load(trained_model, ma

[]
Processing Image: https://m.media-amazon.com/images/I/213wY3gUsmL.jpg
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=500x171 at 0x7E53BCAAD030>
236.9709707602339
['30.6 centimetre', '6 centimetre']
Processing Image: https://m.media-amazon.com/images/I/214CLs1oznL.jpg
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=500x500 at 0x7E54F00AECE0>
247.935904
['43 inch', '10 centimetre']
Processing Image: https://m.media-amazon.com/images/I/214CLs1oznL.jpg
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=500x500 at 0x7E53BC1D55D0>
247.935904
['43 inch', '10 centimetre']
Processing Image: https://m.media-amazon.com/images/I/214CLs1oznL.jpg
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=500x500 at 0x7E53BCAAE860>
247.935904
['43 inch', '10 centimetre']
Appended results to /kaggle/working/output_easyocr.csv
Processing Image: https://m.media-amazon.com/images/I/216rjgJHAeL.jpg
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=175x175 at 0x7E53BCAAE230>
215.5274

...

[]
Processing Image: https://m.media-amazon.com/images/I/31LeiIQqy4L.jpg
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=500x185 at 0x7E54E5F5F220>
240.1220864864865
['3 kilogram']
Processing Image: https://m.media-amazon.com/images/I/31Lfi-0r4GL.jpg
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=500x500 at 0x7E53C77D78E0>
234.701884
['19 centimetre']
Processing Image: https://m.media-amazon.com/images/I/31Lfi-0r4GL.jpg
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=500x500 at 0x7E53C7C401C0>
234.701884
['19 centimetre']
Processing Image: https://m.media-amazon.com/images/I/31LmviTSuzS.jpg
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1024x1024 at 0x7E53C666C9A0>
242.32349967956543
['60 centimetre']
Appended results to /kaggle/working/output_easyocr.csv
Processing Image: https://m.media-amazon.com/images/I/31LmviTSuzS.jpg
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1024x1024 at 0x7E54F00AE8F0>
242.32349967956543


/opt/conda/lib/python3.10/site-packages/easyocr/detection.py:85: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(copyStateDict(torch.load(trained_model, ma

['60 centimetre']
Processing Image: https://m.media-amazon.com/images/I/31LrSRISE0L.jpg
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=500x364 at 0x7E53C77D5780>
201.39872527472528
[]
Processing Image: https://m.media-amazon.com/images/I/31LrSRISE0L.jpg
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=500x364 at 0x7E53BD338100>
201.39872527472528
[]
Processing Image: https://m.media-amazon.com/images/I/31LwkMqi+DL.jpg
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=500x453 at 0x7E53BD656A40>
171.67427814569535
[]
Processing Image: https://m.media-amazon.com/images/I/31LwkMqi+DL.jpg
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=500x453 at 0x7E53D4B2F640>
171.67427814569535
[]
Appended results to /kaggle/working/output_easyocr.csv
Processing Image: https://m.media-amazon.com/images/I/31LwkMqi+DL.jpg
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=500x453 at 0x7E53BD5D5030>
171.67427814569535


/opt/conda/lib/python3.10/site-packages/easyocr/detection.py:85: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(copyStateDict(torch.load(trained_model, ma

[]
Processing Image: https://m.media-amazon.com/images/I/31MG0yf59ZL.jpg
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=393x500 at 0x7E53BE75D330>
173.19065648854962
['100 gram']
Processing Image: https://m.media-amazon.com/images/I/31MLB7TWBVL.jpg
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=138x500 at 0x7E53D4B2F9D0>
185.27195652173913
[]
Processing Image: https://m.media-amazon.com/images/I/31MRd8GBmjL.jpg
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=607x614 at 0x7E53C77D7EE0>
201.4697986036952
[]
Processing Image: https://m.media-amazon.com/images/I/31MSh7g6jHL.jpg
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=500x500 at 0x7E53BD5D4E20>
232.377276
[]
Appended results to /kaggle/working/output_easyocr.csv
Processing Image: https://m.media-amazon.com/images/I/31MUroBHxlL.jpg
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=500x156 at 0x7E53D4B2FE50>
248.48726923076924
